In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
sys.path.insert(0, '../scripts')
from simulation import *

In [12]:
P = 16
N = 80000
x_f = 1.5
q_f = 0.8
theta = 0
sigma = 0.05
tau = 1e-2
dt = 1e-3
p = 0.005
K = N*c
A = 10

In [13]:
patterns = np.random.RandomState(seed=1).normal(0,1,size=(P,N))

In [14]:
# Structural connectivity
k = np.random.RandomState(seed=2).binomial(N, p, size=N)
ji = build_ji(k, N)

100%|██████████| 80000/80000 [01:51<00:00, 717.28it/s]


In [15]:
# Uniform learning rule distribution: symmetric <--> asymmetric
mask_sym = np.random.RandomState(seed=12).rand(N)
mask_asym = 1 - mask_sym

In [18]:
# Store pattern associations using the threshold plasticity rule
plasticity = ThresholdPlasticityRule(x_f, q_f)
indptr, indices, data_p0 = store_associations(
    patterns, plasticity.f, plasticity.g, ji, K, p=0,
    mask=mask_sym)
_, _, data_p1 = store_associations(
    patterns, plasticity.f, plasticity.g, ji, K, p=1,
    mask=mask_asym)

100%|██████████| 80000/80000 [00:07<00:00, 11174.81it/s]


In [19]:
# Adjust subpopulation weights
data_p1_copy = np.asarray(data_p1).copy()
data_p0_copy = np.asarray(data_p0).copy()
reweight(indptr, indices, data_p1_copy, # Asymmetric
         A, N, w_11=1, w_12=1, w_21=1, w_22=1)
reweight(indptr, indices, data_p0_copy, # Symmetric
         A, N, w_11=1, w_12=1, w_21=1, w_22=1)
data = data_p1_copy + data_p0_copy
W = cp.sparse.csr_matrix(
    (cp.asarray(data), cp.asarray(indices), cp.asarray(indptr)),
    shape=(N,N),
    dtype=cp.float32)

80000it [00:00, 145136.02it/s]
80000it [00:00, 123170.06it/s]


In [ ]:
I_ext_1 = np.linspace(-0.5, 0, 31)
I_ext_2 = np.linspace(-0.25, 0.25, 31)

params = {
    "N": N,
    "p": p,
    "K": K,
    "P": P,
    "A": A,
    "tau": tau,
    "theta": theta,
    "sigma": sigma,
    "x_f": plasticity.x_f,
    "q_f": plasticity.q_f,
    "x_g": plasticity.x_g,
    "q_g": plasticity.q_g,
}

combinations = list(itertools.product(
    np.atleast_1d(I_ext_1),
    np.atleast_1d(I_ext_2)))

directory = "5-bilinear-continuum/data/"

try:
    os.makedirs(directory)
except FileExistsError:
    pass

for args in tqdm(combinations[:]):
    params["I_ext_1"] = I_ext_1 = args[0]
    params["I_ext_2"] = I_ext_2 = args[1]
    
    def xi_t(t):
        xi = cp.zeros(N)
        xi += I_ext_1 * cp.asarray(mask_asym)
        xi += I_ext_2 * cp.asarray(mask_sym)
        return xi
    
    r = simulate(
        2.0,  # T
        dt, # dt
        tau, # tau
        N=N,
        I_ext_1=None,
        I_ext_2=None,
        I_ext=xi_t,
        theta=theta,
        sigma=sigma,
        patterns=patterns,
        r0=phi(cp.asarray(patterns[0,:]), theta, sigma),
        W=W,
        disable_pbar=True)
    q, _, _ = correlations(r, patterns, individual=False)
    filename = "Iext1%.6f_Iext2%.6f"%(I_ext_1,I_ext_2) + ".npy"
    filepath = directory + filename
    np.save(open(filepath, 'wb'), {
        "q": q,
        "r": r[:100,:].astype(np.float16),
        "params": params})

100%|██████████| 961/961 [4:07:29<00:00, 15.46s/it]  